In [ ]:
# Load libraries
import numpy as np
import pandas as pd
import io
from google.colab import files
from timeit import default_timer

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving dataset_final.csv to dataset_final.csv


In [ ]:
dat = pd.read_csv(io.BytesIO(uploaded['dataset_final.csv']), sep = ";")
dat

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV,dataset
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0,train
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6,train
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,train
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,train
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,0.00,36.2,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,0.00,22.4,train
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6,test
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9,train
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0,train


In [ ]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
 14  dataset  506 non-null    object 
dtypes: float64(11), int64(3), object(1)
memory usage: 59.4+ KB


In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
categorical_vars = ['RAD']
non_categorical_vars = list(set(dat.columns) - set(categorical_vars))

In [ ]:
# 3) Define "model"
ohe = OneHotEncoder(sparse_output = False, drop = 'first')

# 4) Train "model"
ohe.fit(dat[categorical_vars][dat['dataset'] == 'train'])

# 5) "Predict"
dat_ohe = pd.DataFrame(ohe.transform(dat[categorical_vars]))

# Optional
dat_ohe.columns = ohe.get_feature_names_out()

# Combine numerical and categorical
dat = pd.concat((dat[non_categorical_vars], dat_ohe), axis=1)

In [ ]:
!pip install fancyimpute
from fancyimpute import IterativeImputer as MICE

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.4 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29880 sha256=d194082dda59f85d93a14333966c3c7ce5a3b5bb8d8ea3957b9cc021c3badf04
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=81dec4c85813d65051ffcde8eb90d3125db7972782b87599b24ab41f2a1f2aa4
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [ ]:
numerical_vars = list(set(dat.columns) - set(['CHAS', 'dataset']))

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# 3) Define "model"
model = MICE(DecisionTreeRegressor()) #if needed change this

# 4) Train "model"
model.fit(dat[numerical_vars][dat['dataset'] == 'train'])

# 5) "Predict"
dat[numerical_vars] = model.transform(dat[numerical_vars])

In [ ]:
dat

,B,NOX,RM,dataset,ZN,TAX,DIS,INDUS,PTRATIO,CHAS,...,CRIM,AGE,RAD_2,RAD_3,RAD_4,RAD_5,RAD_6,RAD_7,RAD_8,RAD_24
0,396.90,0.538,6.575,train,18.0,296.0,4.0900,2.31,15.3,0,...,0.00632,65.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,396.90,0.469,6.421,train,0.0,242.0,4.9671,7.07,17.8,0,...,0.02731,78.9,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,392.83,0.469,7.185,train,0.0,242.0,4.9671,7.07,17.8,0,...,0.02729,61.1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,394.63,0.458,6.998,train,0.0,222.0,6.0622,2.18,18.7,0,...,0.03237,45.8,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,396.90,0.458,7.147,test,0.0,222.0,6.0622,2.18,18.7,0,...,0.06905,54.2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,391.99,0.573,6.593,train,0.0,273.0,2.4786,11.93,21.0,0,...,0.06263,69.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502,396.90,0.573,6.120,test,0.0,273.0,2.2875,11.93,21.0,0,...,0.04527,76.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
503,396.90,0.573,6.976,train,0.0,273.0,2.1675,11.93,21.0,0,...,0.06076,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
504,393.45,0.573,6.794,train,0.0,273.0,2.3889,11.93,21.0,0,...,0.10959,89.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   B        506 non-null    float64
 1   NOX      506 non-null    float64
 2   RM       506 non-null    float64
 3   dataset  506 non-null    object 
 4   ZN       506 non-null    float64
 5   TAX      506 non-null    float64
 6   DIS      506 non-null    float64
 7   INDUS    506 non-null    float64
 8   PTRATIO  506 non-null    float64
 9   CHAS     506 non-null    int64  
 10  LSTAT    506 non-null    float64
 11  MEDV     506 non-null    float64
 12  CRIM     506 non-null    float64
 13  AGE      506 non-null    float64
 14  RAD_2    506 non-null    float64
 15  RAD_3    506 non-null    float64
 16  RAD_4    506 non-null    float64
 17  RAD_5    506 non-null    float64
 18  RAD_6    506 non-null    float64
 19  RAD_7    506 non-null    float64
 20  RAD_8    506 non-null    float64
 21  RAD_24   506 non

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
sampling_strategy = 50/50

In [ ]:
# Apply SMOTE
sm = SMOTE(sampling_strategy = sampling_strategy,
      random_state = 0,
      k_neighbors = 5)
X_res, y_res = sm.fit_resample(dat[dat['dataset'] == 'train'].drop(['dataset', 'CHAS'], axis = 1), dat[dat['dataset'] == 'train']['CHAS'])


# Add dataset and exitus columns
X_res['CHAS'] = y_res
X_res['dataset'] = 'train'

# Concatenate new train set after SMOTE with original validation and test sets
dat = pd.concat([X_res, dat[dat['dataset'] == 'val'], dat[dat['dataset'] == 'test']])

In [ ]:
100*dat[dat['dataset'] == 'train'].CHAS.value_counts()/dat[dat['dataset'] == 'train'].shape[0]

0    50.0
1    50.0
Name: CHAS, dtype: float64

In [ ]:
from sklearn.tree import DecisionTreeClassifier as model_constructor

In [ ]:
from sklearn.metrics import f1_score as metric


In [ ]:
criterion_values = ['gini', 'entropy']
max_depth_values = [None, 6, 20];
min_samples_split_values = [2, 5, 20];
min_samples_leaf_values = [1, 5, 20];
max_features_values = [None, 1, 2];

params_grid = {  'criterion': criterion_values,
                 'max_depth': max_depth_values,
                 'min_samples_split': min_samples_split_values,
                 'min_samples_leaf': min_samples_leaf_values,
                 'max_features': max_features_values}

In [ ]:
num_iter = 1;
grid_results = pd.DataFrame(columns = ('criterion',
                                       'max_depth',
                                       'min_samples_split',
                                       'min_samples_leaf',
                                       'max_features',
                                       'metric_train',
                                       'metric_val'))

for criterion in params_grid['criterion']:
    for max_depth in params_grid['max_depth']:
        for min_samples_split in params_grid['min_samples_split']:
            for min_samples_leaf in params_grid['min_samples_leaf']:
                for max_features in params_grid['max_features']:

                    # Print trace
                    print('Iteration = ' + str(num_iter))

                    # [3] Define model
                    model = model_constructor(criterion = criterion,
                                              max_depth = max_depth,
                                              min_samples_split = min_samples_split,
                                              min_samples_leaf = min_samples_leaf,
                                              max_features = max_features,
                                              random_state = 0)

                    # [4] Train model
                    model.fit(dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].CHAS.values)

                    # [5] Predict

                    #accuracy
                    pred_train = model.predict(dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1)) # predict_proba!
                    pred_val = model.predict(dat[dat['dataset'] == 'val'].drop(['CHAS', 'dataset'], axis = 1))

                    # [6] Evaluate
                    metric_train = metric(dat[dat['dataset'] == 'train'].CHAS.values, pred_train)
                    metric_val = metric(dat[dat['dataset'] == 'val'].CHAS.values, pred_val)

                    # print error
                    print('AUC train = %.2f - AUC validation = %.2f.'
                          % (metric_train, metric_val))

                    # Save iteration results
                    grid_results.loc[num_iter]=[criterion,
                                                max_depth,
                                                min_samples_split,
                                                min_samples_leaf,
                                                max_features,
                                             metric_train,
                                             metric_val]
                    num_iter += 1

Iteration = 1
AUC train = 1.00 - AUC validation = 0.00.
Iteration = 2
AUC train = 1.00 - AUC validation = 0.20.
Iteration = 3
AUC train = 1.00 - AUC validation = 0.00.
Iteration = 4
AUC train = 0.97 - AUC validation = 0.00.
Iteration = 5
AUC train = 0.93 - AUC validation = 0.50.
Iteration = 6
AUC train = 0.96 - AUC validation = 0.00.
Iteration = 7
AUC train = 0.92 - AUC validation = 0.00.
Iteration = 8
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 9
AUC train = 0.80 - AUC validation = 0.09.
Iteration = 10
AUC train = 0.99 - AUC validation = 0.00.
Iteration = 11
AUC train = 0.98 - AUC validation = 0.00.
Iteration = 12
AUC train = 0.98 - AUC validation = 0.00.
Iteration = 13
AUC train = 0.97 - AUC validation = 0.00.
Iteration = 14
AUC train = 0.93 - AUC validation = 0.50.
Iteration = 15
AUC train = 0.96 - AUC validation = 0.00.
Iteration = 16
AUC train = 0.92 - AUC validation = 0.00.
Iteration = 17
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 18
AUC train = 0.80 - AUC va

In [ ]:
grid_results_dt = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model_dt = grid_results_dt.iloc[0]
best_model_dt

criterion                gini
max_depth                None
min_samples_split           2
min_samples_leaf            5
max_features                1
metric_train         0.927622
metric_val                0.5
Name: 5, dtype: object

In [ ]:
# [3] define model
model = model_constructor(criterion = best_model_dt.criterion,
                          max_depth = best_model_dt.max_depth,
                          min_samples_split = int(best_model_dt.min_samples_split),
                          min_samples_leaf = int(best_model_dt.min_samples_leaf),
                          max_features = best_model_dt.max_features,
                          random_state = 0)

# [4] Train model
model.fit(dat[dat['dataset'] != 'test'].drop(['CHAS', 'dataset'], axis = 1), dat[dat['dataset'] != 'test'].CHAS.values)


# [5] Predict

#accuracy
pred_train = model.predict(dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1)) # predict!
pred_val = model.predict(dat[dat['dataset'] == 'val'].drop(['CHAS', 'dataset'], axis = 1)) # predict!
pred_test = model.predict(dat[dat['dataset'] == 'test'].drop(['CHAS', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].CHAS.values, pred_train)
metric_val = metric(dat[dat['dataset'] == 'val'].CHAS.values, pred_val)
metric_test = metric(dat[dat['dataset'] == 'test'].CHAS.values, pred_test)

# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.93 - Metric val = 0.18 - Metric test = 0.35


In [ ]:
from xgboost import XGBClassifier as model_constructor

In [ ]:
# Xgboost
n_estimators_values = [1000]
learning_rate_values = [1]
gamma_values = [1, 10]
max_depth_values = [6, 20]
min_child_weight_values = [20, 100]
subsample_values = [0.1, 1]
colsample_bytree_values = [0.1, 1]
num_parallel_tree_values = [10]

params_grid = {'n_estimators': n_estimators_values,
                  'learning_rate': learning_rate_values,
                 'gamma': gamma_values,
                 'max_depth': max_depth_values,
                 'min_child_weight': min_child_weight_values,
                 'subsample': subsample_values,
                 'colsample_bytree': colsample_bytree_values,
                 'num_parallel_tree': num_parallel_tree_values}

In [ ]:
num_iter = 1;
grid_results = pd.DataFrame(columns = ('n_estimators',
                                       'learning_rate',
                                       'gamma',
                                       'max_depth',
                                       'min_child_weight',
                                       'subsample',
                                       'colsample_bytree',
                                       'num_parallel_tree',
                                       'best_iteration',
                                       'metric_train',
                                       'metric_val'))

for n_estimators in params_grid['n_estimators']:
    for learning_rate in params_grid['learning_rate']:
        for gamma in params_grid['gamma']:
            for max_depth in params_grid['max_depth']:
                for min_child_weight in params_grid['min_child_weight']:
                    for subsample in params_grid['subsample']:
                        for colsample_bytree in params_grid['colsample_bytree']:
                            for num_parallel_tree in params_grid['num_parallel_tree']:




                                                # Print trace
                                                print('Iteration = ' + str(num_iter))

                                                # [3] Define model
                                                model = model_constructor(n_estimators = n_estimators,
                                                                      learning_rate = learning_rate,
                                                                      gamma = gamma,
                                                                      max_depth = max_depth,
                                                                      min_child_weight = min_child_weight ,
                                                                      subsample = subsample,
                                                                      colsample_bytree = colsample_bytree,
                                                                      num_parallel_tree = num_parallel_tree,
                                                                      early_stopping_rounds = 10,
                                                                      eval_metric = "logloss",
                                                                      random_state = 0) # nthread!!!

                                                # [4] Train model
                                                model.fit(dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1), dat[dat['dataset'] == 'train'].CHAS.values,
                                                          eval_set=[(dat[dat['dataset'] == 'val'].drop(['CHAS', 'dataset'], axis = 1), dat[dat['dataset'] == 'val'].CHAS.values)],
                                                          verbose = False)
                                                best_iteration = model.best_iteration

                                                # [5] Predict
                                                #accuracy
                                                pred_train = model.predict(dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1))
                                                pred_val = model.predict(dat[dat['dataset'] == 'val'].drop(['CHAS', 'dataset'], axis = 1))

                                                # [6] Compute metric
                                                metric_train = metric(dat[dat['dataset'] == 'train'].CHAS.values, pred_train)
                                                metric_val = metric(dat[dat['dataset'] == 'val'].CHAS.values, pred_val)

                                                # print error
                                                print('AUC train = %.2f - AUC validation = %.2f.'
                                                      % (metric_train, metric_val))

                                                # Save iteration results
                                                grid_results.loc[num_iter]=[n_estimators,
                                                                            learning_rate,
                                                                            gamma,
                                                                            max_depth,
                                                                            min_child_weight,
                                                                            subsample,
                                                                            colsample_bytree,
                                                                            num_parallel_tree,
                                                                            best_iteration,
                                                                            metric_train,
                                                                            metric_val]
                                                num_iter += 1

Iteration = 1
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 2
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 3
AUC train = 0.95 - AUC validation = 0.29.
Iteration = 4
AUC train = 0.95 - AUC validation = 0.44.
Iteration = 5
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 6
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 7
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 8
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 9
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 10
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 11
AUC train = 0.95 - AUC validation = 0.29.
Iteration = 12
AUC train = 0.95 - AUC validation = 0.44.
Iteration = 13
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 14
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 15
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 16
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 17
AUC train = 0.00 - AUC validation = 0.00.
Iteration = 18
AUC train = 0.00 - AUC va

In [ ]:
grid_results_xgb = grid_results.sort_values(by = ['metric_val', 'metric_train'], ascending = [False, False])
best_model_xgb = grid_results_xgb.iloc[0]
best_model_xgb

n_estimators         1000.000000
learning_rate           1.000000
gamma                   1.000000
max_depth               6.000000
min_child_weight       20.000000
subsample               1.000000
colsample_bytree        1.000000
num_parallel_tree      10.000000
best_iteration         21.000000
metric_train            0.945827
metric_val              0.444444
Name: 4, dtype: float64

In [ ]:
# [3] define model
model = model_constructor(n_estimators = int(best_model_xgb.best_iteration),
                          learning_rate = best_model_xgb.learning_rate,
                          gamma = best_model_xgb.gamma,
                          max_depth = int(best_model_xgb.max_depth),
                          min_child_weight = best_model_xgb.min_child_weight,
                          subsample = best_model_xgb.subsample,
                          colsample_bytree = best_model_xgb.colsample_bytree,
                          num_parallel_tree = int(best_model_xgb.num_parallel_tree),
                          random_state = 0)

# [4] Train model
model.fit(dat[dat['dataset'] != 'test'].drop(['CHAS', 'dataset'], axis = 1), dat[dat['dataset'] != 'test'].CHAS.values)


# [5] Predict
#accuracy
pred_train = model.predict(dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1)) # predict!
pred_val = model.predict(dat[dat['dataset'] == 'val'].drop(['CHAS', 'dataset'], axis = 1)) # predict!
pred_test = model.predict(dat[dat['dataset'] == 'test'].drop(['CHAS', 'dataset'], axis = 1)) # predict!


# [6] Compute metric
metric_train = metric(dat[dat['dataset'] == 'train'].CHAS.values, pred_train)
metric_val = metric(dat[dat['dataset'] == 'val'].CHAS.values, pred_val)
metric_test = metric(dat[dat['dataset'] == 'test'].CHAS.values, pred_test)

#accuracy binary class
#metric_train = metric(dat[dat['dataset'] == 'train'].exitus.values, pred_train)
#metric_val = metric(dat[dat['dataset'] == 'val'].exitus.values, pred_val)
#metric_test = metric(dat[dat['dataset'] == 'test'].exitus.values, pred_test)

# print error
print('Metric train = %.2f - Metric val = %.2f - Metric test = %.2f'
      % (metric_train, metric_val, metric_test))

Metric train = 0.95 - Metric val = 0.67 - Metric test = 0.37


In [ ]:
model

n_estimators         1000.000000
learning_rate           1.000000
gamma                   1.000000
max_depth               6.000000
min_child_weight       20.000000
subsample               1.000000
colsample_bytree        1.000000
num_parallel_tree      10.000000
best_iteration         21.000000
metric_train            0.945827
metric_val              0.444444
Name: 4, dtype: float64

In [ ]:
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier as model_constructor

model = model_constructor(n_estimators = int(best_model_xgb.best_iteration),
                          learning_rate = best_model_xgb.learning_rate,
                          gamma = best_model_xgb.gamma,
                          max_depth = int(best_model_xgb.max_depth),
                          min_child_weight = best_model_xgb.min_child_weight,
                          subsample = best_model_xgb.subsample,
                          colsample_bytree = best_model_xgb.colsample_bytree,
                          num_parallel_tree = int(best_model_xgb.num_parallel_tree),
                          random_state = 0)


# 4) Train the model
model.fit(dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1),
                                  dat[dat['dataset'] == 'train'].CHAS.values)

# perform permutation importance
importance = permutation_importance(model,
                                    dat[dat['dataset'] == 'val'].drop(['CHAS', 'dataset'], axis = 1),
                                    dat[dat['dataset'] == 'val'].CHAS.values,
                                    random_state = 0)
X = dat[dat['dataset'] == 'train'].drop(['CHAS', 'dataset'], axis = 1)

# get importance
importance = pd.DataFrame({'variable' : X.columns.values, 'imp' : np.abs(importance.importances_mean)/np.max(np.abs(importance.importances_mean))}).sort_values(by = 'imp', ascending = False)
importance

,variable,imp
14,RAD_4,1.000000
8,LSTAT,0.956522
2,RM,0.826087
4,TAX,0.826087
6,INDUS,0.826087
9,MEDV,0.782609
0,B,0.652174
7,PTRATIO,0.565217
5,DIS,0.347826
1,NOX,0.217391


In [ ]:
grid_results_dt

,criterion,max_depth,min_samples_split,min_samples_leaf,max_features,metric_train,metric_val
5,gini,None,2,5,1,0.927622,0.5
14,gini,None,5,5,1,0.927622,0.5
59,gini,20,2,5,1,0.927622,0.5
68,gini,20,5,5,1,0.927622,0.5
30,gini,6,2,1,2,0.828619,0.5
...,...,...,...,...,...,...,...
125,entropy,6,5,20,1,0.000000,0.0
134,entropy,6,20,20,1,0.000000,0.0
143,entropy,20,2,20,1,0.000000,0.0
152,entropy,20,5,20,1,0.000000,0.0


In [ ]:
grid_results_xgb

,n_estimators,learning_rate,gamma,max_depth,min_child_weight,subsample,colsample_bytree,num_parallel_tree,best_iteration,metric_train,metric_val
4,1000.0,1.0,1.0,6.0,20.0,1.0,1.0,10.0,21.0,0.945827,0.444444
12,1000.0,1.0,1.0,20.0,20.0,1.0,1.0,10.0,21.0,0.945827,0.444444
3,1000.0,1.0,1.0,6.0,20.0,1.0,0.1,10.0,34.0,0.948454,0.285714
11,1000.0,1.0,1.0,20.0,20.0,1.0,0.1,10.0,34.0,0.948454,0.285714
20,1000.0,1.0,10.0,6.0,20.0,1.0,1.0,10.0,2.0,0.917511,0.266667
28,1000.0,1.0,10.0,20.0,20.0,1.0,1.0,10.0,2.0,0.917511,0.266667
19,1000.0,1.0,10.0,6.0,20.0,1.0,0.1,10.0,13.0,0.933333,0.000000
27,1000.0,1.0,10.0,20.0,20.0,1.0,0.1,10.0,13.0,0.933333,0.000000
1,1000.0,1.0,1.0,6.0,20.0,0.1,0.1,10.0,21.0,0.000000,0.000000
2,1000.0,1.0,1.0,6.0,20.0,0.1,1.0,10.0,21.0,0.000000,0.000000
